In [1]:
try:
    import psycopg2, pydal, attrs, tabulate
    del psycopg2, pydal, attrs, tabulate
    print('requirements satisfied')
except ImportError as e:
    print(e)
    print('installing requiremens')
    !pip install psycopg2-binary pydal attrs tabulate

requirements satisfied


In [2]:
def readable(v):
    if isinstance(v, uuid.UUID):
        return str(v)[:4]
    try:
        # als het een enum betreft dan pak de waarde
        return str(v.value)[:2].capitalize()
    except: pass
    try:
        return str(v.time())
    except: pass
    try:
        if v.count('_') > 0:
            return ''.join([part[0].upper() for part in v.split('_')])
    except:
        pass
    return v


def show(title=None, rows=None, tablefmt='github'):
    if rows is None:
        rows, title = title, rows

    rowlist = [{readable(k):readable(v) for k,v in row.items() if not callable(v)} for row in rows]
    print(rowlist)
    table = tabulate(rowlist, headers='keys', tablefmt=tablefmt)
    if title:
        width = len(table.split('\n')[0])
        dashes = width - len(title) - 4
        c1 = dashes // 2
        c2 = dashes - c1
        print(f'{"-"*c1}[ {title} ]{"-"*c2}')
    print(table)
    print()

In [3]:
import edwh.core.sync
import importlib
importlib.reload(edwh.core.sync)
from edwh.core.sync import *
#!pip install psycopg2-binary pydal attrs tabulate
# import datetime, time
# import re
# import uuid
# from enum import Enum, auto
# from pprint import pprint
#
# from attrs import define, field, asdict
# from pydal import DAL, Field, SQLCustomType
# from pydal.validators import IS_IN_SET, IS_EMAIL, IS_MATCH
# from tabulate import tabulate
#
# RE_EMAIL = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')
# RE_DOMAIN = re.compile(r'[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+')
# RE_UUID = re.compile(r'[0-9a-fA-F]{8}\b-[0-9a-fA-F]{4}\b-[0-9a-fA-F]{4}\b-[0-9a-fA-F]{4}\b-[0-9a-fA-F]{12}')
#
# class Status(Enum):
#     DRAFT = 'draft'
#     PROPOSED = 'proposed'
#     PUBLISHED = 'published'
#     # REJECTED = 'rejected'
#     HIDDEN = 'hidden'
#     RETRACTED = 'retracted'
#
# class Role(Enum):
#     LOCAL_EDITOR=auto()
#     TEAM=auto()
#     DOMAIN=auto()
#     PUBLIC=auto()
#     CENTRAL_EDITOR=auto()
#
# values = lambda enum_items: {s.value for s in enum_items}
# CENTRAL_EDITOR_QUERY_STATUSES = values(Status)
# LOCAL_EDITOR_QUERY_STATUSES = values({Status.DRAFT, Status.PROPOSED, Status.PUBLISHED, Status.RETRACTED}) # later uit te breiden met Status.hidden
# TEAM_QUERY_STATUSES = values({Status.PROPOSED, Status.PUBLISHED, Status.RETRACTED}) # later uit te breiden met Status.hidden
# DOMAIN_QUERY_STATUSES = values({Status.PROPOSED, Status.PUBLISHED, Status.RETRACTED}) # later mogelijk uit te breiden met Status.hidden
# PUBLIC_QUERY_STATUSES = values({Status.PUBLISHED, Status.RETRACTED}) # later mogelijk uit te breiden met Status.hidden
#
# visible_statuses_per_role = {
#     Role.LOCAL_EDITOR:LOCAL_EDITOR_QUERY_STATUSES,
#     Role.TEAM:TEAM_QUERY_STATUSES,
#     Role.DOMAIN:DOMAIN_QUERY_STATUSES,
#     Role.PUBLIC:PUBLIC_QUERY_STATUSES,
#     Role.CENTRAL_EDITOR:CENTRAL_EDITOR_QUERY_STATUSES
# }
#
# status_dal_type = SQLCustomType(type='string', native='text', encoder=lambda status:status.value, decoder=lambda fromdb:Status(fromdb))
# uuid_dal_type = SQLCustomType(type='string', native='uuid', encoder=lambda id:str(id), decoder=lambda fromdb:uuid.UUID(fromdb), represent=lambda _uuid:str(_uuid)[:5]+'+')
# datetime_dal_type = SQLCustomType(type='datetime')
#
# class Scope(Enum):
#     EDITOR=auto()
#     TEAM=auto()
#     DOMAIN=auto()
#     PUBLIC=auto()
#
# @define
# class State:
#     gid: uuid.UUID = field()
#     editor_status: Status = field()
#     team_status: Status = field()
#     domain_status: Status = field()
#     public_status: Status = field()
#     by: str = field()
#     origin_domain: str = field()
#
#     # remove microsecond part since this isn't stored in the database. And it's useless for our case.
#     ts: datetime.datetime = field(factory=lambda:datetime.datetime.now().replace(microsecond=0))
#     transaction_gid: uuid.UUID = field(factory=uuid.uuid4)
#
#     @by.validator
#     def check_email(self, attribute, value):
#         if not RE_EMAIL.fullmatch(value):
#             raise ValueError(f'Invalid email format {value!r}')
#
#
# def REQUIRED(fieldname):
#     def wrapped():
#         raise ValueError(f'Value missing for field {fieldname}')
#     return wrapped
#
#
# @define
# class SomeState(State):
#     __tablename = 'some'
#     value:str = field(default=REQUIRED('value')) # required is a workaround to force a default argument
#
#     @classmethod
#     def from_db(cls, db, row):
#         return cls(**{field:row[field] for field in db[cls.__tablename].fields if field != 'id'})
#
#     @classmethod
#     def from_row(cls, row:DAL.Row):
#         return cls(
#               gid=row.gid,
#               editor_status=row.editor_status,
#               team_status=row.team_status,
#               domain_status=row.domain_status,
#               public_status=row.public_status,
#               by=row.by,
#               origin_domain=row.origin_domain,
#               ts=row.ts,
#               transaction_gid=row.transaction_gid,
#               value=row.value,
#         )
#
#     @classmethod
#     def most_recent(cls, db:DAL, as_role:Role, domain:str, gid:uuid.UUID = None, debug=True):
#         """Dit moet de meest recente rij opgegeven binnen de opgegeven belemmeringen"""
#         # TODO: domain wordt nu nog niet gebruikt, en de vraag is of dat nodig is.
#         # immers een instantie mag alleen maar krijgen wat bij het eigen domein afkomstig is
#         # of wat is aangeboden vanuit landelijk.
#         field_for_role = {
#             Role.LOCAL_EDITOR:'editor_status',
#             Role.TEAM:'team_status',
#             Role.DOMAIN:'domain_status',
#             Role.PUBLIC:'public_status',
#             Role.CENTRAL_EDITOR:'editor_status'
#         }
#
#
#         table = db[cls.__tablename]
#         table_rname = table._rname
#         show(str(table),db(table).select())
#         # if given a gid, it should be considered, otherwise all the relevant rows should be returned
#         gid_clause = f"{table_rname}.gid = '{str(gid)}'" if gid else '(true)'
#         # as the central editor role, domain is irrelevant as a criteria
#         origin_clause = f"{table_rname}.origin_domain = '{domain}'" if as_role != Role.CENTRAL_EDITOR else 'true'
#
#         # this subselect should return all the id's for the unique rows which should
#         # be valid for the given the creteria. The fields are then selected given these id's (to avoid huge group by clauses)
#         # for performance reasons this might turn into a CTE, but that's premature optimization for now.
#         # the most accurate row can yield a HIDDEN status for the respective scope status.
#         # Hidden rows are filtered in the query which consumes this one.
#         # basically it's the effective dated rows, based on the domain and gid selection
#         # If we would include the hidden check in here, we would filter out all the hidden rows, but that might be
#         # effective status for this selection, so that filtering has to be done a layer up.
#         # the where filters on BOTH gid and domain, because the status can be different for the same gid
#         # across the different domains.
#         subselect = f'''
#             select id
#               from {table_rname}
#              where
#                    {gid_clause}
#                    and ts = (
#                     select max(ts)
#                       from {table_rname} as effdted
#                       where effdted.gid = {table_rname}.gid
#                         and effdted.origin_domain = {table_rname}.origin_domain
#                         and {field_for_role[as_role]} in ({", ".join(f"'{_}'" for _ in visible_statuses_per_role[as_role])})
#                    ) and ({origin_clause})
#              group by id
#         '''
#         if debug:
#             print('subselect: ')
#             print(subselect)
#             print('subselect result: ')
#             print(db.executesql(subselect))
#
#         # the selection of fields and the filtering on the scope_status field being anything but 'hidden'
#         sql = f'''
#         select *
#           from {table_rname}
#          where id in (
#             {subselect}
#          )
#         '''
#         if debug:
#             print('complete query:')
#             print(sql)
#         result = (cls.from_row(row) for row in db.executesql(sql, fields=table))
#         if debug:
#             result_list = list(result)
#             print('result:')
#             pprint(result_list)
#             # returna  new generator
#             return (_ for _ in result_list)
#         else:
#             return result
#
#
# def define_table(db:DAL):
#     state_table = db.Table(db, 'state',
#         Field('gid',uuid_dal_type,comment='The GID of the object for which this is a state.'),
#         Field('editor_status',status_dal_type,requires=IS_IN_SET(set(s.value for s in Status))),
#         Field('team_status',status_dal_type,requires=IS_IN_SET(set(s.value for s in Status))),
#         Field('domain_status',status_dal_type,requires=IS_IN_SET(set(s.value for s in Status))),
#         Field('public_status',status_dal_type,requires=IS_IN_SET(set(s.value for s in Status))),
#         Field('by','string',requires=IS_EMAIL),
#         Field('origin_domain','string', requires=IS_MATCH(RE_DOMAIN.pattern)),
#         Field('ts','datetime', default=lambda:datetime.datetime.now().replace(microsecond=0)),
#         Field('transaction_gid',uuid_dal_type, requires=IS_MATCH(RE_UUID.pattern), default=lambda:str(uuid.uuid4())),
#     )


In [4]:
import os
server = os.getenv('POSTGRES_SERVER', '10.221.142.201')
db = DAL(f'postgres://username:password@{server}/db')
db.define_table('some',
    state_table(db),
    Field('value','text'),
    rname='some_tbl'
)
db.some.truncate()
db.commit()

In [5]:
CURRENT_DOMAIN = 'remco.nl'
SECOND_DOMAIN='second.nl'
some_gid = uuid.uuid4()
other_gid = uuid.uuid4()

In [6]:
db['some']._rname

'some_tbl'

In [7]:
db.some.insert(**asdict(first_edit := SomeState(
    some_gid,
    Status.DRAFT,
    Status.HIDDEN,
    Status.HIDDEN,
    Status.HIDDEN,
    'remco@ewdh.nl',
    CURRENT_DOMAIN,
    value='A) 1st domain, 1st edit'
)))

def first_test():
    assert list(SomeState.most_recent(db, Role.LOCAL_EDITOR, domain=CURRENT_DOMAIN, gid=some_gid))[0] == first_edit, 'editor should see his own edit'
    assert list(SomeState.most_recent(db, Role.PUBLIC, domain=CURRENT_DOMAIN, gid=some_gid)) == [], 'public should not see this edit yet'
first_test()

In [8]:
time.sleep(1)
db.some.insert(**asdict(second_edit := SomeState(
    some_gid,
    Status.DRAFT,
    Status.HIDDEN,
    Status.HIDDEN,
    Status.HIDDEN,
    'remco@ewdh.nl',
    CURRENT_DOMAIN,
    value='A) 2nd draft, 1st domain'
)))
def second_test():
    assert len(list(SomeState.most_recent(db, Role.LOCAL_EDITOR, domain=CURRENT_DOMAIN, gid=some_gid))) == 1, 'only one should be shown for this user'
    assert list(SomeState.most_recent(db, Role.LOCAL_EDITOR, domain=CURRENT_DOMAIN, gid=some_gid))[0] == second_edit, 'editor should see his own second edit'
    assert list(SomeState.most_recent(db, Role.PUBLIC, domain=CURRENT_DOMAIN, gid=some_gid)) == [], 'public should not see this edit yet'
second_test()

In [9]:
db.some.insert(**asdict(third_edit := SomeState(
    other_gid,
    Status.DRAFT,
    Status.HIDDEN,
    Status.HIDDEN,
    Status.HIDDEN,
    'remco@ewdh.nl',
    CURRENT_DOMAIN,
    value='B) 1st edition, 3rd edit')
))
time.sleep(1)

# kopie tests voor other_gid

def third_test():
    assert list(SomeState.most_recent(db, Role.LOCAL_EDITOR, domain=CURRENT_DOMAIN, gid=other_gid)) == [third_edit], 'editor should see his own edit'
    assert list(SomeState.most_recent(db, Role.PUBLIC, domain=CURRENT_DOMAIN, gid=other_gid)) == [], 'public should not see this edit yet'
second_test()
third_test()



In [10]:
db.some.insert(**asdict(second_domain_first:=SomeState(
    some_gid,
    Status.DRAFT,
    Status.HIDDEN,
    Status.HIDDEN,
    Status.HIDDEN,
    'other@second.nl',
    SECOND_DOMAIN,
    value='A) 2nd domain, 1st edit'
)))
time.sleep(1)

third_test()
def second_domain_test():
    assert list(SomeState.most_recent(db, Role.LOCAL_EDITOR, domain=SECOND_DOMAIN, gid=some_gid))[0] == second_domain_first, 'editor should see his own edit'
    assert list(SomeState.most_recent(db, Role.LOCAL_EDITOR, domain=SECOND_DOMAIN))[0] == second_domain_first, 'editor should see his own edit'
    assert list(SomeState.most_recent(db, Role.PUBLIC, domain=CURRENT_DOMAIN, gid=other_gid)) == [], 'public should not see this edit yet'
    assert list(SomeState.most_recent(db, Role.PUBLIC, domain=CURRENT_DOMAIN)) == [], 'public should not see this edit yet'
second_domain_test()

In [11]:
db.commit()
db.some.insert(**asdict(fourth_state := SomeState(
    some_gid,
    Status.PROPOSED,
    Status.HIDDEN,
    Status.HIDDEN,
    Status.HIDDEN,
    'remco@ewdh.nl',
    CURRENT_DOMAIN,
    value='A) 1st domain, 1st proposal'
)))
time.sleep(1)

db.commit()

# unique index afdwingen vereist ivm duplicaten issues
TODO: er moet een sleutel toegevoegd worden in de database om te voorkomen dat er voor 1 object binnen 1 tijdstempel binnen hetzelfde domein verschillende items plaatsvinden, omdat dan niet na te gaan is wat de meest recente/relevante is.

Aan de andere kant is dit ook op te lossen door een unieke sleutel af te dwingen in de database on insert, aangzien het gaat over de database van 1 platform.


In [12]:
db.rollback()
def fourth_test():
    assert list(SomeState.most_recent(db, Role.LOCAL_EDITOR, domain=CURRENT_DOMAIN, gid=some_gid))[0] == fourth_state, 'editor should see his own proposal'
    assert list(SomeState.most_recent(db, Role.PUBLIC, domain=CURRENT_DOMAIN, gid=some_gid)) == [], 'public should not see this edit yet'
    assert fourth_state in list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None, gid=some_gid)), 'public editors should see this propoasal, without domain'
    assert list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None, gid=some_gid)) == [second_domain_first, fourth_state], 'public editors should see this proposal, for both domains, but only for the given gid.'
    assert list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None))==[third_edit, second_domain_first, fourth_state] , 'public editors should see the proposal and the draft for B, when no domain nor gid are given'
# fourth_state in list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None, gid=some_gid))
fourth_test()


# Rejected wel of niet?
Rejected is een vreemde eend in de bijt. eigenlijk wil je vanuit een landelijk platform wel zien of je een wijziging accepteerd, maar je wilt deze niet doorgeven aan anderen.

Op basis van de [RFC procedure van het IETF](https://www.rfc-editor.org/rfc/rfc2026.html) zou ik zeggen dat een rejected niet nodig is, en niet als status wordt vastgelegd. Een draft wordt genegeerd als deze niet gecommiteerd/gepubliceerd wordt. Een historic status zoals zij deze kennen is bij ons niet relevant, omdat de queries er voor moeten zorgen dat de laatste versie naar boven komen. Documenten die elkaar overerven kennen we niet in onze structuur.

# Retracted
Als er geen rejected komt, kan een document ook nooit verwijderd worden. Dus daarvoor is er de status "Retracted", waarbij het artikel voor niemand meer zichtbaar is. De body mag de meldig zijn die dan op het scherm verschijnt, zodat je ook aan een eindgebruiker eventueel kunt vertellen waarom dit document niet meer bestaat, danwel waar je verder moet kijken.

In [13]:
# if False:
#     db.some.insert(**asdict(SomeState(
#         some_gid,
#         Status.REJECTED,
#         Status.HIDDEN,
#         Status.HIDDEN,
#         Status.HIDDEN,
#         'eddie@ewdh.nl',
#         CURRENT_DOMAIN,
#         value='A) rejected'
#     )))
#     time.sleep(1)
#
#     db.some.insert(**asdict(SomeState(
#         some_gid,
#         Status.DRAFT,
#         Status.HIDDEN,
#         Status.HIDDEN,
#         Status.HIDDEN,
#         'remco@ewdh.nl',
#         CURRENT_DOMAIN,
#         value='A) draft na reject'
#     )))
#     time.sleep(1)
#
#     db.some.insert(**asdict(SomeState(
#         some_gid,
#         Status.PROPOSED,
#         Status.HIDDEN,
#         Status.HIDDEN,
#         Status.HIDDEN,
#         'remco@ewdh.nl',
#         CURRENT_DOMAIN,
#         value='A) propose na draft na reject'
#     )))
#     time.sleep(1)

db.some.insert(**asdict(fifth_state :=SomeState(
    some_gid,
    Status.PUBLISHED,
    Status.PUBLISHED,
    Status.PUBLISHED,
    Status.PUBLISHED,
    'remco@ewdh.nl',
    CURRENT_DOMAIN,
    value='A) published na propose na draft na reject'
)))
time.sleep(1)
db.commit()

In [14]:
db.rollback()
def fifth_test():
    assert list(SomeState.most_recent(db, Role.LOCAL_EDITOR, domain=CURRENT_DOMAIN, gid=some_gid)) == [fifth_state], 'editor should see the published version'
    assert list(SomeState.most_recent(db, Role.PUBLIC, domain=CURRENT_DOMAIN, gid=some_gid)) == [fifth_state], 'public should see the published version'
    assert fifth_state in list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None, gid=some_gid)), 'public editors should see this published version, without domain'
    assert list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None, gid=some_gid)) == [second_domain_first, fifth_state], 'public editors should see this published version, for both domains, but only for the given gid.'
    assert list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None))==[third_edit, second_domain_first, fifth_state] , 'public editors should see the proposal and the draft for B, when no domain nor gid are given'
# fourth_state in list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None, gid=some_gid))
fifth_test()

In [15]:
db.some.insert(**asdict(sixth_state :=SomeState(
    some_gid,
    Status.DRAFT,
    Status.HIDDEN,
    Status.HIDDEN,
    Status.HIDDEN,
    'remco@ewdh.nl',
    CURRENT_DOMAIN,
    value='A) draft na published'
)))
time.sleep(1)
db.commit()

In [16]:
db.rollback()
def sixth_test():
    assert list(SomeState.most_recent(db, Role.LOCAL_EDITOR, domain=CURRENT_DOMAIN, gid=some_gid)) == [sixth_state], 'editor should see the latest draft'
    assert list(SomeState.most_recent(db, Role.PUBLIC, domain=CURRENT_DOMAIN, gid=some_gid)) == [fifth_state], 'public should see the last published version'
    assert sixth_state in list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None, gid=some_gid)), 'public editors should see this published version, without domain'
    assert list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None, gid=some_gid)) == [second_domain_first, sixth_state], 'central editors should see this draft, for both domains, but only for the given gid.'
    assert list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None))==[third_edit, second_domain_first, sixth_state] , 'public editors should see the proposal and the draft for B, when no domain nor gid are given'
# fourth_state in list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None, gid=some_gid))
sixth_test()

In [17]:
db.some.insert(**asdict(seventh_state := SomeState(
    some_gid,
    Status.RETRACTED,
    Status.RETRACTED,
    Status.RETRACTED,
    Status.RETRACTED,
    'remco@ewdh.nl',
    CURRENT_DOMAIN,
    value='A) is ingetrokken'
)))
time.sleep(1)
db.commit()

In [18]:

db.rollback()

def seventh_test():
    assert list(SomeState.most_recent(db, Role.LOCAL_EDITOR, domain=CURRENT_DOMAIN, gid=some_gid)) == [
        seventh_state], 'editor should see the latest draft'
    assert list(SomeState.most_recent(db, Role.PUBLIC, domain=CURRENT_DOMAIN, gid=some_gid)) == [
        seventh_state], 'public should see the last published version'
    assert seventh_state in list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None,
                                                     gid=some_gid)), 'public editors should see this published version, without domain'
    assert list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None, gid=some_gid)) == [second_domain_first,
                                                                                               seventh_state], 'public editors should see this draft, for both domains, but only for the given gid.'
    assert list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None)) == [third_edit, second_domain_first,
                                                                                 seventh_state], 'public editors should see the proposal and the draft for B, when no domain nor gid are given'


# fourth_state in list(SomeState.most_recent(db, Role.CENTRAL_EDITOR, domain=None, gid=some_gid))
seventh_test()

In [19]:
row = db(db.some).select().last()
s2 = SomeState.from_db(db, row )

In [20]:
for k,v in {field:row[field] for field in db.some.fields}.items():
    print(f'  {k}=row.{k}')

  id=row.id
  gid=row.gid
  editor_status=row.editor_status
  team_status=row.team_status
  domain_status=row.domain_status
  public_status=row.public_status
  by=row.by
  origin_domain=row.origin_domain
  ts=row.ts
  transaction_gid=row.transaction_gid
  value=row.value


In [21]:
s2

SomeState(gid=UUID('cc9330eb-b5cc-41a8-8acf-6edd1468a535'), editor_status=<Status.RETRACTED: 'retracted'>, team_status=<Status.RETRACTED: 'retracted'>, domain_status=<Status.RETRACTED: 'retracted'>, public_status=<Status.RETRACTED: 'retracted'>, by='remco@ewdh.nl', origin_domain='remco.nl', ts=datetime.datetime(2022, 10, 20, 22, 6, 55), transaction_gid=UUID('8e6c7e78-d46a-436d-bcd1-d3da58e2b5c8'), value='A) is ingetrokken')